<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/scientific-computing-examples/blob/main/notebooks/batch_create_a2_vm.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank"  href="https://raw.githubusercontent.com/GoogleCloudPlatform/scientific-computing-examples/main/notebooks/batch_create_a2_vm.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
</table>

# Overview
This tutorial will demonstrate how to run some simple jobs using [Google Cloud Batch](https://cloud.google.com/batch/docs/get-started). A few key points:

 1. You have the choice of using a [project you have already created](https://cloud.google.com/resource-manager/docs/creating-managing-projects), or creating one here.
 1. You use a [existing GCS bucket](https://cloud.google.com/storage/docs/creating-buckets#create_a_new_bucket) from your existing project, or you  can create a new one here.
 1. Most of the work communicating with Google Cloud is done with the [gcloud command](https://cloud.google.com/sdk/gcloud)
 1. There are no VMs to provision. All that is done automatically for you by Batch.

The following series of steps will take you through some of the basics of using Cloud Batch.

 > **IMPORTANT:** 
 
It is assumed that the system hosting this notebook has `gcloud` installed. If not, [it can be installed according to these instructions](https://cloud.google.com/sdk/docs/install)
  For this reason, we recommend the use of [Google Colab](https://research.google.com/colaboratory/)



# Getting Started
There are several steps to configure your settings with Google Cloud. Except for the Python installation below, all are completed with the `gcloud` command.
## Run Pip to install packages
Some Python packages are not preinstalled. We add them here.

In [ ]:
%%capture
#@title Pip Install Packages: Run Me! {display-mode: "form"}
!pip install Jinja2
!pip3 install grpcio

## Build Input UI. 

The form below will is for you to enter the required (and optional) information to make the rest of the tutorial work. You will enter:

1. [The project ID](https://cloud.google.com/resource-manager/docs/creating-managing-projects#before_you_begin)
1. You user ID, which is the login id you used to connect to the Google Cloud Console.
1. The [region where you want the jobs](https://cloud.google.com/compute/docs/regions-zones) to run.
1. The name of you [GCS Bucket](https://cloud.google.com/storage/docs/creating-buckets), either already created or the one to be created in your project.
1. Your [billing id](https://cloud.google.com/billing/docs/how-to/view-linked), you may have to ask your organization manager. This is required only if you plan to create a new project here.

Run the next code cell to create the data entry form.

In [ ]:
#@title Form Setup: Run Me! {display-mode: "form"}
#@markdown Run this section to enter your information into the notebook

import ipywidgets as widgets
import subprocess
from IPython.core.display import display, Markdown

account = subprocess.run(['gcloud', 'config', 'get-value', 'account'], stdout=subprocess.PIPE)
if not account.stdout: 
   account.stdout = 'enter-user-id' 

project = subprocess.run(['gcloud', 'config', 'get-value', 'project'], stdout=subprocess.PIPE)
if not project.stdout: 
   project.stdout = 'enter-project-id' 

region = subprocess.run(['gcloud',  'config',  'get-value',  'compute/region'],  stdout=subprocess.PIPE)
if not region.stdout: 
   region.stdout = 'us-central1' 

bucket_name = "enter-bucket-name"
billing_id = "enter-billing-id"


project = widgets.Text( value=project.stdout, disabled=False )
username = widgets.Text( value=account.stdout, disabled=False )
region = widgets.Text( value=region.stdout, disabled=False )
bucket = widgets.Text( value=bucket_name, disabled=False )
billing = widgets.Text( value=billing_id, disabled=False )



accordion = widgets.Accordion(children=[project, username, region, bucket, billing])
accordion.set_title(0, 'ProjectID')
accordion.set_title(1, 'UserID')
accordion.set_title(2, 'Region')
accordion.set_title(3, 'Bucket Name')
accordion.set_title(4, 'Billing ID [Optional]')
accordion.selected_index = None

display(Markdown("## Enter values below: "))
display(accordion)


## Authenticate to Google Cloud
This step allows you to connect and run `gcloud` commands. If you get a <span style="color:yellow">"WARNING"</span>, you may already be logged in.

Follow the instructions to ensure the login process is completed.


In [ ]:
!gcloud auth login $username.value

## Create the project
If the billing ID and the Project ID are set above, you can click here to create the project. If the project already exists, you will get a warning, but it will still work.


In [ ]:
def create_project():
    proj = project.value.rstrip("\n\r ") ## remove "\n", "\r" char in the project value var

    # Don't do project creationg if the billing ID is not set
    if billing.value == "enter-billing-id":
      print("Billing ID not set")
    else:
      !gcloud projects create $proj
      !gcloud beta billing projects link $proj --billing-account=$billing.value
      
create_project()

In [ ]:
def create_bucket():
    proj = project.value.rstrip("\n\r ") ## remove "\n", "\r" char in the project value var
    buck = bucket.value.rstrip("\r\n ") ## remove "\n" char in the bucket value var

    !echo "Executing: "  gcloud alpha storage buckets create gs://$buck --project=$proj
    !gcloud alpha storage buckets create gs://$buck --project=$proj
      
create_bucket()

## Create gcloud configuration

There are two things happening here:
1. You are setting the configurations for `gcloud` defaults
1. You are enabling Google Cloud APIs
1. Finally, the full configuration is listed out.

It is important that these happen in the right order, for example, you have to enable the `compute` API before `gcloud` lets you set a variable for `compute`.

This code cell takes about 30 seconds to execute.

In [ ]:
# set project
!gcloud config set project $project.value 
#enable APIs
!gcloud services enable compute.googleapis.com 
!gcloud services enable batch.googleapis.com
!gcloud services enable monitoring.googleapis.com
# set region 
!gcloud config set compute/region $region.value

# list out the complete configuation
!gcloud config list

## Create _Instance Template_ for Batch to create a VM
The following command creates an instance template that is used by Batch to
create a VM.

In [ ]:
!gcloud compute instance-templates create gpu-schedmd  \
   --machine-type=a2-highgpu-2g   \
   --maintenance-policy=TERMINATE    \
    --image-family=schedmd-v5-slurm-22-05-3-hpc-centos-7   \
    --image-project=schedmd-slurm-public    \
     --boot-disk-size=300

In [ ]:
#@title Generate buttons to submit jobs {display-mode: "form"}
#@markdown Create a job by selecting the button.
import jinja2
import sys
from os import environ
import ipywidgets as widgets
import uuid

jobid = ""
environment = jinja2.Environment()


def create_jobtypelist():
    jobtypelist = {}
   
    gpu_template = """
    {
        "taskGroups":[
            {
            "taskSpec":{
                "runnables": [
          
		            {
                    "script": {
                    "text": "nvidia-smi >> /mnt/share/log_${JOB_ID}_${BATCH_TASK_INDEX}.txt"
                    },
                    "environment": {
                        "variables": {
                          "JOB_ID": "{{ job_id }} "
                        }
                    }
                  } 
                ],
                "volumes": [
                {
                    "gcs": {
                    "remotePath": "{{ bucket_name }}"
                    },
                    "mountPath": "/mnt/share"
                }
                ]
            },
            "taskCount": 3
            }
        ],
        "allocationPolicy": {
            "location": {
               "allowed_locations": [ "regions/{{ region_name }}" ]
             },
            "instances": [
		    {
			"instanceTemplate": "gpu-schedmd"
		    }
            ]
        },
        "labels": {
            "department": "wdc",
            "env": "testing"
        },
        "logsPolicy": {
            "destination": "CLOUD_LOGGING"
        }
    }

    """


    jobtypelist["gpu"] = {"desc": "Create a2_a100 VM ", "code" : gpu_template}
    return(jobtypelist)

jobtypelist = create_jobtypelist()

class MyButton(widgets.Button):
    code = ""

def run_job(b):
    jobid = "job-" + uuid.uuid4().hex[:8]
    template = environment.from_string(b.code)
    render = template.render(bucket_name=bucket.value, 
                             job_id=jobid, 
                             region_name=region.value)
    f = open(jobid + ".json", "w")
    f.write(render)
    f.close()
    out2.clear_output()
    with out2:
        print("\n\nCreating job")
        print("\n\nCreating job file: ", jobid + "txt")
        jobstring = !echo gcloud beta batch jobs submit {jobid} --location {region.value} --config {jobid}.json
        print("Executing command: " + jobstring[0])
        # print(render)
        
        ## This is the command line that is being executed.
        !gcloud beta batch jobs submit {jobid} --location {region.value} --config {jobid}.json


out = widgets.Output(layout={'border': '1px solid black'})
display(out)
out2 = widgets.Output(layout={'border': '1px solid black'})
display(out2)
with out:
    for jobtype in jobtypelist:
        button = MyButton(description=jobtypelist[jobtype]["desc"],  button_style='success')
        button.code = jobtypelist[jobtype] = jobtypelist[jobtype]["code"]
        display(button)
        button.on_click(run_job)



In [ ]:
#@title You can see the list of jobs in the GCP Console {display-mode: "form"}

from IPython.core.display import display, Markdown
display(Markdown("[View Job list in the Console](https://console.cloud.google.com/batch/jobs?project=" + project.value + ")"))

## See output of the "GCS" job
If you ran the "GCS" job, the output was sent to the bucket you created above. 
 
 The next cell provides a sample of how to see the output of all the outputs stored in the GCS bucket.

> The output should simply be the output of the "nvdia-smi" command.

In [ ]:
!gcloud storage ls gs://$bucket.value/*.txt
!gcloud storage cat gs://$bucket.value/*.txt